In [ ]:
!ls ../input/100-bird-species/consolidated

In [ ]:
!ls ../input/100-bird-species/

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transform
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
transform = transform.Compose([transform.RandomHorizontalFlip(),
                                transform.RandomRotation(0.2),
                                transform.ToTensor(),
                                transform.Resize((80,80))
                               ])

In [ ]:
train= torchvision.datasets.ImageFolder(root="../input/100-bird-species/train", transform=transform)
val= torchvision.datasets.ImageFolder(root="../input/100-bird-species/valid", transform=transform)
test= torchvision.datasets.ImageFolder(root="../input/100-bird-species/test", transform=transform)

In [ ]:
print("length of training data is {} and length of validation data is {}".format(len(train), len(val)))

In [ ]:
print(val[224][0][1])

In [ ]:
plt.imshow(val[0][224][224])

In [ ]:
val[0]

In [ ]:
plt.figure(figsize=(12,12))
for i in range(224,228):
    plt.imshow(val[i][0])

In [ ]:
plt.figure(figsize=(12,12))

#subplot(r,c) provide the no. of rows and columns
_, axarr = plt.subplots(4,1) 

# use the created array to output your multiple images. In this case I have stacked 4 images vertically
axarr[0].imshow(val[224][0])
axarr[1].imshow(val[225][0])
axarr[2].imshow(val[226][0])
axarr[3].imshow(val[227][0])

In [ ]:
val[224][1]

In [ ]:
def plotImages(images):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    #axes = axes.flatten()
    for img, ax in zip(images, axes):
        ax.imshow(img)
        ax.axis('off')
    #plt.tight_layout()
    plt.show()
    
images = [val[224+i][0] for i in range(10)]
plotImages(images)

In [ ]:
cuda = False
if torch.cuda.is_available():
    cuda = True

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(5*5*128, 1600)
        self.fc2 = nn.Linear(1600, 3200)
        self.fc3 = nn.Linear(3200, 260)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [ ]:
model = Net()
if cuda==True:
    model.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
num_epochs=10

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train,batch_size=32,shuffle=True)
valid_data_loader = torch.utils.data.DataLoader(val,batch_size=32,shuffle=True)
test_data_loader = torch.utils.data.DataLoader(test,batch_size=32,shuffle=False)

In [ ]:
device= torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = Net().to(device)

In [ ]:
total_step = len(train_data_loader)
for epoch in range(num_epochs):
    model.train()
    training_accuracy=0
    val_accuracy=0
    for i, (images, labels) in enumerate(train_data_loader):
        
        images = images.cuda()
        labels = labels.cuda()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        # Checking accuracy
        preds = outputs.data.max(dim = 1, keepdim = True)[1]
        training_accuracy += preds.eq(labels.data.view_as(preds)).cpu().sum()
        
        
    training_accuracy = training_accuracy/len(train_data_loader.dataset) * 100
        
    print ('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {}' 
                   .format(epoch+1, num_epochs, loss.item(), training_accuracy))


In [ ]:
# Train the model
model.train()  # train mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Train Accuracy of the model on the 8000 test images with Relu is: {} %'.format(100 * correct / total))


In [ ]:
total_step = len(test_data_loader)
for epoch in range(num_epochs):
    model.eval()
    test_accuracy=0
    val_accuracy=0
    for i, (images, labels) in enumerate(test_data_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        # Checking accuracy
        preds = outputs.data.max(dim = 1, keepdim = True)[1]
        test_accuracy += preds.eq(labels.data.view_as(preds)).cpu().sum()
        
        
    test_accuracy = test_accuracy/len(test_data_loader.dataset) * 100
        
    print ('Epoch [{}/{}], Loss: {:.4f}, Testing Accuracy: {}' 
                   .format(epoch+1, num_epochs, loss.item(), test_accuracy))


In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 8000 test images with Relu is: {} %'.format(100 * correct / total))


In [ ]:
vgg = torchvision.models.vgg11(pretrained=True)
vgg

In [ ]:
vgg.classifier[6].out_features = 260
for param in vgg.features.parameters(): 
    param.requires_grad = False

vgg = vgg.cuda()

In [ ]:
total_step = len(train_data_loader)
for epoch in range(num_epochs):
    vgg.train()
    training_accuracy=0
    val_accuracy=0
    for i, (images, labels) in enumerate(train_data_loader):
        
        images = images.cuda()
        labels = labels.cuda()
        
        # Forward pass
        outputs = vgg(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        # Checking accuracy
        preds = outputs.data.max(dim = 1, keepdim = True)[1]
        training_accuracy += preds.eq(labels.data.view_as(preds)).cpu().sum()
        
        
    training_accuracy = training_accuracy/len(train_data_loader.dataset) * 100
        
    print ('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {}' 
                   .format(epoch+1, num_epochs, loss.item(), training_accuracy))


In [ ]:
# Train the model
vgg.train()  # train mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Train Accuracy of the model on the 8000 test images with Relu is: {} %'.format(100 * correct / total))


In [ ]:
total_step = len(test_data_loader)
for epoch in range(num_epochs):
    vgg.eval()
    test_accuracy=0
    val_accuracy=0
    for i, (images, labels) in enumerate(test_data_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = vgg(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        # Checking accuracy
        preds = outputs.data.max(dim = 1, keepdim = True)[1]
        test_accuracy += preds.eq(labels.data.view_as(preds)).cpu().sum()
        
        
    test_accuracy = test_accuracy/len(test_data_loader.dataset) * 100
        
    print ('Epoch [{}/{}], Loss: {:.4f}, Testing Accuracy: {}' 
                   .format(epoch+1, num_epochs, loss.item(), test_accuracy))


In [ ]:
# Test the model
vgg.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = vgg(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 8000 test images with Relu is: {} %'.format(100 * correct / total))
